In [1]:
import requests
import pandas as pd
import json

In [2]:
def get_api(base_url="https://zucwflxqsxrsmwseehqvjmnx2u0cdigp.lambda-url.ap-south-1.on.aws",end_point="/mentorskool/v1/sales?offset=0&limit=100",headers={"access_token":"fe66583bfe5185048c66571293e0d358"}):
    """
    This function loops through the api to fetch all the records and return it in a list.This function takes all types of errors into account.
    :param base_url:Base url of the api
           endpoint:endpoint for fetching the record from that page
           headers: contains the access token for authorization
    :return: A list conatining all the data
    """
    final_list=[]
    for i in range(5):
        try:

            response = requests.get(base_url+end_point,headers=headers)

            response_data = response.json()
            data=response_data['data']
            final_list.extend(data)
            end_point= response_data['next']
        except requests.exceptions.HTTPError as http_err:
            print(f"HTTPError:",{http_err})
        except Exception as err:
            print(f"Error:", {err})
    return final_list

In [3]:
whole_list = get_api()

In [5]:
sales_df = pd.json_normalize(whole_list)

In [6]:
sales_df.dtypes

id                                       int64
sales_amt                              float64
qty                                      int64
discount                               float64
profit_amt                              object
order.order_id                          object
order.ship_mode                         object
order.order_status                      object
order.order_purchase_date               object
order.order_approved_at                 object
order.order_delivered_carrier_date      object
order.order_delivered_customer_date     object
order.order_estimated_delivery_date     object
order.customer.customer_id              object
order.customer.customer_name            object
order.customer.segment                  object
order.customer.contact_number           object
order.customer.address.zip_code          int64
order.customer.address.region           object
order.customer.address.country          object
order.customer.address.city             object
order.custome

In [7]:
sales_df.replace('null',None,inplace=True)
sales_df.isnull().sum()

id                                       0
sales_amt                                0
qty                                      0
discount                                 0
profit_amt                               1
order.order_id                           0
order.ship_mode                          0
order.order_status                       0
order.order_purchase_date                0
order.order_approved_at                  2
order.order_delivered_carrier_date       7
order.order_delivered_customer_date     12
order.order_estimated_delivery_date      2
order.customer.customer_id               0
order.customer.customer_name             0
order.customer.segment                   0
order.customer.contact_number            0
order.customer.address.zip_code          0
order.customer.address.region            0
order.customer.address.country           0
order.customer.address.city              0
order.customer.address.state             0
order.vendor.VendorID                    0
order.vendo

In [8]:
sales_df['order.order_purchase_date']=pd.to_datetime(sales_df['order.order_purchase_date'])
sales_df['order.order_approved_at']=pd.to_datetime(sales_df['order.order_approved_at'])
sales_df['order.order_delivered_carrier_date']=pd.to_datetime(sales_df['order.order_delivered_carrier_date'])
sales_df['order.order_delivered_customer_date']=pd.to_datetime(sales_df['order.order_delivered_customer_date'])
sales_df['order.order_estimated_delivery_date']=pd.to_datetime(sales_df['order.order_estimated_delivery_date'])

In [9]:
sales_df.columns

Index(['id', 'sales_amt', 'qty', 'discount', 'profit_amt', 'order.order_id',
       'order.ship_mode', 'order.order_status', 'order.order_purchase_date',
       'order.order_approved_at', 'order.order_delivered_carrier_date',
       'order.order_delivered_customer_date',
       'order.order_estimated_delivery_date', 'order.customer.customer_id',
       'order.customer.customer_name', 'order.customer.segment',
       'order.customer.contact_number', 'order.customer.address.zip_code',
       'order.customer.address.region', 'order.customer.address.country',
       'order.customer.address.city', 'order.customer.address.state',
       'order.vendor.VendorID', 'order.vendor.Vendor Name',
       'product.product_id', 'product.product_name', 'product.colors',
       'product.category', 'product.sub_category', 'product.date_added',
       'product.manufacturer', 'product.sizes', 'product.upc',
       'product.weight', 'product.product_photos_qty'],
      dtype='object')

In [10]:
def product_size(df,product):
    product_sizes= df[df['product.product_name']==product]['product.sizes'].unique()
    return product_sizes

In [11]:
a = product_size(sales_df,'Mitel 5320 IP Phone VoIP phone')
a

array([None], dtype=object)

which month had the highest sales overall?

In [12]:
sales_df['month'] = sales_df['order.order_purchase_date'].apply(lambda date:date.strftime('%B'))


In [13]:
monthly_sales=sales_df.groupby('month')['sales_amt'].sum()
print('highest sale by month:',monthly_sales.idxmax())

highest sale by month: May


which month had the highest overall profit?

In [22]:
monthly_profit = sales_df.groupby('month')['profit_amt'].sum()
monthly_profit.sort_values()

month
June        -1297.0189
May          -832.6445
December     -617.9495
August       -350.6432
September    -281.3286
July           -84.115
April         -56.0825
November      365.0812
January       905.8689
October      1130.7502
February     1329.5155
March        5129.2084
Name: profit_amt, dtype: object

how many months have lead to a positive profit margin?

In [15]:
num_positive_months=(monthly_profit>0).sum()
print('No. of months with positive monthly profit:',num_positive_months)

No. of months with positive monthly profit: 5


In [23]:
sales_df['delay']=  (sales_df['order.order_delivered_customer_date']-sales_df['order.order_estimated_delivery_date']).dt.days

In [24]:
sales_df['delay_status'] = sales_df['delay'].apply(lambda x: 'Late' if x>0 else('Early' if x<0 else 'On Time'))

how many orders are late delivered to the customers?

In [25]:
sales_df['delay_status'].value_counts()

delay_status
Early      452
Late        29
On Time     19
Name: count, dtype: int64

which vendor has the highest late deliveries?

In [28]:
late_df = sales_df[sales_df['delay_status']=='Late']

In [29]:
vendor_late_delivery_count = late_df.groupby('order.vendor.VendorID')['id'].count()
print('vendor with most late deliveries:',vendor_late_delivery_count.idxmax())

vendor with most late deliveries: VEN03


In [30]:
vendor_late_delivery_count

order.vendor.VendorID
VEN01    3
VEN02    7
VEN03    8
VEN04    7
VEN05    4
Name: id, dtype: int64